## Librairies Importations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from scipy.stats import gaussian_kde

%pip install lightgbm
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix, accuracy_score, recall_score, precision_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
%pip install xgboost
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

from tqdm.notebook import tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\Flooz\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\Flooz\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Dataset and visualization

In [4]:
dataset_train_path = 'dataset_part_1/dataset/spotify_dataset_train.csv'
dataset_test_path = 'dataset_part_1/dataset/spotify_dataset_test.csv'

dataset_train = pd.read_csv(dataset_train_path)
dataset_test = pd.read_csv(dataset_test_path)

In [5]:
if dataset_train.isnull().sum().sum() == 0:
    print("No missing values")
else:
    print("Missing values for:")
    print(dataset_train.isnull().sum())

No missing values


In [6]:
# Verifying duplicates in the dataset if needed
print(dataset_train.duplicated().value_counts())
# False = Uniques
# True = Duplicates
#dataset_test.drop_duplicates(inplace=True)

print("="*40)
print("Dataset shape:", dataset_train.shape)
print("="*40)

False    25491
True         1
Name: count, dtype: int64
Dataset shape: (25492, 17)


In [7]:
print("="*40)
dataset_test.info()
print("="*40)

dataset_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2833 entries, 0 to 2832
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   release_date      2833 non-null   object 
 1   explicit          2833 non-null   bool   
 2   popularity        2833 non-null   int64  
 3   danceability      2833 non-null   float64
 4   energy            2833 non-null   float64
 5   key               2833 non-null   int64  
 6   loudness          2833 non-null   float64
 7   mode              2833 non-null   int64  
 8   speechiness       2833 non-null   float64
 9   acousticness      2833 non-null   float64
 10  instrumentalness  2833 non-null   float64
 11  liveness          2833 non-null   float64
 12  valence           2833 non-null   float64
 13  tempo             2833 non-null   float64
 14  duration_ms       2833 non-null   int64  
 15  time_signature    2833 non-null   int64  
dtypes: bool(1), float64(9), int64(5), object(1

,release_date,explicit,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,2018-03-16,True,83,0.699,0.304,1,-11.192,1,0.0993,0.77900,0.000993,0.1370,0.271,119.705,160184,4
1,2003,False,44,0.590,0.722,4,-5.783,1,0.0265,0.00052,0.000009,0.0712,0.670,116.035,166645,4
2,2012-01-01,False,0,0.362,0.435,2,-7.241,1,0.0280,0.27400,0.000001,0.1580,0.349,141.970,220693,4
3,2001-10-23,False,57,0.571,0.617,2,-5.032,1,0.0348,0.32800,0.000000,0.1150,0.574,88.154,187373,4
4,2020-09-18,False,56,0.578,0.748,0,-5.609,0,0.0460,0.03260,0.002550,0.1430,0.170,123.973,185806,4


In [8]:
dataset_train.describe()

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,25492.000000,25492.000000,25492.000000,25492.000000,25492.000000,25492.000000,25492.000000,25492.000000,25492.000000,25492.000000,25492.000000,25492.000000,2.549200e+04,25492.000000
mean,31.185784,0.600785,0.666671,5.319630,-7.827989,0.617841,0.098590,0.247458,0.121762,0.194426,0.517368,122.020427,2.357300e+05,3.949004
std,26.647985,0.167052,0.223217,3.577066,4.190793,0.485925,0.100955,0.298900,0.264749,0.160047,0.252768,28.000996,9.605836e+04,0.313024
min,0.000000,0.000000,0.000020,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.008290,0.000000,0.000000,1.344000e+04,0.000000
25%,0.000000,0.494000,0.526000,2.000000,-9.618250,0.000000,0.038000,0.012900,0.000000,0.093475,0.318000,99.996000,1.833298e+05,4.000000
50%,33.000000,0.613000,0.700000,5.000000,-6.944500,1.000000,0.054400,0.098400,0.000099,0.129000,0.518000,123.130000,2.179200e+05,4.000000
75%,54.000000,0.723000,0.846000,8.000000,-5.047750,1.000000,0.111000,0.415000,0.031700,0.254000,0.721000,139.936000,2.656698e+05,4.000000
max,98.000000,0.988000,1.000000,11.000000,1.869000,1.000000,0.934000,0.996000,0.989000,0.988000,0.988000,244.613000,2.516987e+06,5.000000


## Graphics

https://www.kaggle.com/code/varunsaikanuri/spotify-data-visualization

In [ ]:
fig = make_subplots(rows=3, cols=3, subplot_titles=(
    '<i>popularity', '<i>danceability', '<i>energy', 
    '<i>loudness', '<i>speechiness', '<i>acousticness', 
    '<i>liveness', '<i>valence', '<i>tempo'))

fig.add_trace(go.Histogram(x=dataset_train['popularity'], name='popularity'), row=1, col=1)
fig.add_trace(go.Histogram(x=dataset_train['danceability'], name='danceability'), row=1, col=2)
fig.add_trace(go.Histogram(x=dataset_train['energy'], name='energy'), row=1, col=3)
fig.add_trace(go.Histogram(x=dataset_train['loudness'], name='loudness'), row=2, col=1)
fig.add_trace(go.Histogram(x=dataset_train['speechiness'], name='speechiness'), row=2, col=2)
fig.add_trace(go.Histogram(x=dataset_train['acousticness'], name='acousticness'), row=2, col=3)
fig.add_trace(go.Histogram(x=dataset_train['liveness'], name='liveness'), row=3, col=1)
fig.add_trace(go.Histogram(x=dataset_train['valence'], name='valence'), row=3, col=2)
fig.add_trace(go.Histogram(x=dataset_train['tempo'], name='tempo'), row=3, col=3)


fig.update_xaxes(title_text="Score (0-100)", row=1, col=1)      
fig.update_xaxes(title_text="Score (0-1)", row=1, col=2)         
fig.update_xaxes(title_text="Score (0-1)", row=1, col=3)         
fig.update_xaxes(title_text="dB", row=2, col=1)                  
fig.update_xaxes(title_text="Score (0-1)", row=2, col=2)         
fig.update_xaxes(title_text="Score (0-1)", row=2, col=3)         
fig.update_xaxes(title_text="Score (0-1)", row=3, col=1)         
fig.update_xaxes(title_text="Score (0-1)", row=3, col=2)         
fig.update_xaxes(title_text="BPM", row=3, col=3)                 

fig.update_layout(height=900, width=900, title_text='<b>Feature Distribution')
fig.update_layout(template='plotly_dark', title_x=0.5)

In [ ]:
fig = make_subplots(rows=3, cols=1, subplot_titles=(
    '<i>Speechiness vs Popularity', '<i>Energy vs Danceability', '<i>Energy vs Loudness'))

# Densités
d1 = gaussian_kde(np.vstack([dataset_train['speechiness'], dataset_train['popularity']]))(np.vstack([dataset_train['speechiness'], dataset_train['popularity']]))
d2 = gaussian_kde(np.vstack([dataset_train['energy'], dataset_train['danceability']]))(np.vstack([dataset_train['energy'], dataset_train['danceability']]))
d3 = gaussian_kde(np.vstack([dataset_train['energy'], dataset_train['loudness']]))(np.vstack([dataset_train['energy'], dataset_train['loudness']]))

fig.add_trace(go.Scatter(x=dataset_train['speechiness'], y=dataset_train['popularity'], mode='markers', marker=dict(color=d1, colorscale='Plasma', size=3, opacity=0.6), name='Speech vs Pop'), row=1, col=1)
fig.add_trace(go.Scatter(x=dataset_train['energy'], y=dataset_train['danceability'], mode='markers', marker=dict(color=d2, colorscale='Plotly3', size=3, opacity=0.6), name='Energy vs Dance'), row=2, col=1)
fig.add_trace(go.Scatter(x=dataset_train['energy'], y=dataset_train['loudness'], mode='markers', marker=dict(color=d3, colorscale='Plasma', size=3, opacity=0.6), name='Energy vs Loud'), row=3, col=1)

fig.update_xaxes(title_text='speechiness', row=1, col=1)
fig.update_xaxes(title_text='energy', row=2, col=1)
fig.update_xaxes(title_text='energy', row=3, col=1)
fig.update_yaxes(title_text='popularity', row=1, col=1)
fig.update_yaxes(title_text='danceability', row=2, col=1)
fig.update_yaxes(title_text='loudness', row=3, col=1)

fig.update_layout(height=900, width=800, template='plotly_dark', title_text='<b>Feature Correlations (coloré par densité)', title_x=0.5, showlegend=False)

## Classification - LGBM

In [ ]:
features = ['popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 
            'speechiness', 'acousticness', 'instrumentalness', 'liveness', 
            'valence', 'tempo', 'duration_ms', 'time_signature']

X, y = dataset_train[features], dataset_train['genre']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraînement LightGBM sur le split
LGBM = LGBMClassifier(n_estimators=500, random_state=42, verbose=-1,)
LGBM.fit(X_train, y_train)

# Évaluation sur validation (du train)
y_pred_val = LGBM.predict(X_val)

In [65]:
print("="*40)
print(f"F1 Score (micro): {f1_score(y_val, y_pred_val, average='micro'):.4f}")
print('='*40)
print("Accuracy : ", accuracy_score(y_val, y_pred_val))
print('='*40)

F1 Score (micro): 0.4317
Accuracy :  0.4316532653461463


In [ ]:
cm = confusion_matrix(y_val, y_pred_val)
px.imshow(cm, x=sorted(y_val.unique()), y=sorted(y_val.unique()), text_auto=True, 
          color_continuous_scale='Plasma', template='plotly_dark',
          width=800, height=700)

## Classification - SVM

In [30]:
svm = SVC(kernel='rbf', random_state=42)
svm.fit(X_train, y_train)

y_pred_val_svm = svm.predict(X_val)

In [43]:
print("="*40)
print(f"SVM F1 Score (micro): {f1_score(y_val, y_pred_val_svm, average='micro'):.4f}")
print('='*40)
print("Accuracy : ", accuracy_score(y_val, y_pred_val_svm))
print('='*40)

SVM F1 Score (micro): 0.2161
Accuracy :  0.21612080800156894


In [ ]:
cm_svm = confusion_matrix(y_val, y_pred_val_svm)
px.imshow(cm_svm, x=sorted(y_val.unique()), y=sorted(y_val.unique()), text_auto=True, 
          color_continuous_scale='Plasma', template='plotly_dark')


## Classification - RandomForest

In [44]:
RandomForest = RandomForestClassifier(n_estimators=100, random_state=42)
RandomForest.fit(X_train, y_train)

y_pred_val_rf = RandomForest.predict(X_val)

In [45]:
print("="*40)
print(f"SVM F1 Score (micro): {f1_score(y_val, y_pred_val_svm, average='micro'):.4f}")
print('='*40)
print("Accuracy : ", accuracy_score(y_val, y_pred_val_svm))
print('='*40)

SVM F1 Score (micro): 0.2161
Accuracy :  0.21612080800156894


In [ ]:
cm_rf = confusion_matrix(y_val, y_pred_val_rf)
px.imshow(cm_rf, x=sorted(y_val.unique()), y=sorted(y_val.unique()), text_auto=True, 
          color_continuous_scale='Plasma', template='plotly_dark', width=800, height=700)

## Classification - XGBoost

In [ ]:
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc = le.transform(y_val)

xgb_model = XGBClassifier(n_estimators=100, random_state=42, verbosity=0)
xgb_model.fit(X_train, y_train_enc)

y_pred_val_xgb = le.inverse_transform(xgb_model.predict(X_val))

In [88]:
print("="*40)
print(f"XGBoost F1 Score (micro): {f1_score(y_val, y_pred_val_xgb, average='micro'):.4f}")
print('='*40)
print("Accuracy : ", accuracy_score(y_val, y_pred_val_xgb))
print('='*40)

XGBoost F1 Score (micro): 0.4350
Accuracy :  0.43498725240243186


In [50]:
cm_xgb = confusion_matrix(y_val, y_pred_val_xgb)
px.imshow(cm_xgb, x=sorted(y_val.unique()), y=sorted(y_val.unique()), text_auto=True, 
          color_continuous_scale='Plasma', template='plotly_dark',
          width=800, height=700)

## Data analysis

In [ ]:
dataset_train_subset_path = 'dataset_part_1/dataset/spotify_dataset_subset.csv'
dataset_subset = pd.read_csv(dataset_train_subset_path)

genre_dummies = pd.get_dummies(dataset_subset['genre'], prefix='genre')
dataset_subset = pd.concat([dataset_subset, genre_dummies], axis=1)

features_reg = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature'] + genre_dummies.columns.tolist()

dataset_subset.columns = dataset_subset.columns.str.replace('[', '', regex=True).str.replace(']', '', regex=True).str.replace('<', '', regex=True)
features_reg = [f.replace('[', '').replace(']', '').replace('<', '') for f in features_reg]

X_sub = dataset_subset[features_reg]
y_sub = dataset_subset['popularity']

xgb_reg = XGBRegressor()
xgb_reg.fit(X_sub, y_sub)

y_pred_popularity = xgb_reg.predict(X_sub)
dataset_subset['predicted_popularity'] = y_pred_popularity

dataset_subset.to_csv('predictions_subset.csv', index=False)

ValueError: feature_names must be string, and may not contain [, ] or <

In [109]:
print("="*40)
print(f"R² Score: {xgb_reg.score(X_sub, y_sub):.4f}")
print('='*40)
print(f"MAE: {np.mean(np.abs(y_sub - y_pred_popularity)):.4f}")
print('='*40)

R² Score: 0.7893
MAE: 9.2980
